In [2]:
import math

In [175]:
"""
Constant Names for CFG
"""
S = "S" #sentence
NP = "NP" #noun phrase
VP = "VP" #verb phrase
PP = "PP"
D = "D" #determiner
N = "N" #noun
V = "V" #verb
P = "P" #pronoun

In [176]:
"""
example CFG for initial testing
i.e. S : ([(NP,VP)],0.5) => S -> NP VP with a probability of 0.5
"""
grammar = {
    S : [((NP,VP),.8),((VP, NP),.2)],
    NP: [((D,N), .8), ((NP,NP), .2) ],
    VP: [((V,P), 1)],
    D : [(("the"), 1)],
    N : [(("woman"), .6),(("man"), .4)],
    V : [(("saw"), 1)],
    P : [(("him"), 1)]
}

grammar2 = {
    S : [((NP,VP),1)],
    NP: [ ((D,N), .6), ((NP,PP), .3), (("Papa"), .1) ],
    VP: [((V,NP), .6), ((VP,PP), .4)],
    PP :[((P,NP), 1)],
    D : [(("the"), .5), (("a"), .5)],
    N : [(("caviar"), .6),(("spoon"), .4)],
    V : [(("spoon"), .1),(("ate"), .9)],
    P : [(("with"), 1)]
}

grammar3 = {
    S : [((NP,VP),1)],
    NP: [ ((NP,PP), .4), (("astronomers"), .1), (("ears"), .18), (("saw"), .04), (("stars"), .18), (("telescopes"), .1) ],
    VP: [((V,NP), .7), ((VP,PP), .3)],
    PP :[((P,NP), 1)],
    V : [(("saw"), 1)],
    P : [(("with"), 1)]
}

In [183]:
for key, value in grammar2.items():
    print(key, " -> ", value)
    print()

S  ->  [(('NP', 'VP'), 1)]

NP  ->  [(('D', 'N'), 0.6), (('NP', 'PP'), 0.3), ('Papa', 0.1)]

VP  ->  [(('V', 'NP'), 0.6), (('VP', 'PP'), 0.4)]

PP  ->  [(('P', 'NP'), 1)]

D  ->  [('the', 0.5), ('a', 0.5)]

N  ->  [('caviar', 0.6), ('spoon', 0.4)]

V  ->  [('spoon', 0.1), ('ate', 0.9)]

P  ->  [('with', 1)]



In [177]:
"""
example sentences
"""

sentence = "the woman saw him"
sentence03 = "astronomers saw stars with ears"
sentence04 = "Papa ate the caviar with a spoon"

In [180]:
def inside(sentence, grammar):
    """
    Perform the inside algorithm

    :param sentence: The sentence to parse
    :param grammar: A dictionary representing the CNF grammar, with the key being the LHS and the value being a list of possible RHS.
    :return: A nxn CKY Table, with each cell containing the max possible log probability
    """
    #split sentence into words
    words = sentence.split()
    n = len(words)
    
    # Initialize the table to be nxn
    inside_table = [[{} for j in range(n)] for i in range(n)]
    
    # Fill in the diagonals of the table
    for i, word in enumerate(words):
        for lhs, rules in grammar.items():
            for rhs, prob in rules:
                if rhs == (word):
                    inside_table[i][i][lhs] = math.log(prob)

    for j in range(2, n + 1):
        for i in range(n - j + 1):
            for k in range(1, j):
                for lhs, rules in grammar.items():
                    for rhs, prob in rules:
                        if len(rhs) == 2:
                            if rhs[0] in inside_table[i][i + k - 1] and rhs[1] in inside_table[i + k][i + j - 1]:

                                new_prob = inside_table[i][i + k - 1][rhs[0]] + inside_table[i + k][i + j - 1][rhs[1]] + math.log(prob)
                                
                                if lhs in inside_table[i][i + j - 1]:
                                    inside_table[i][i + j - 1][lhs] += new_prob
                                else:
                                    inside_table[i][i + j - 1][lhs] = new_prob

    return inside_table
    

In [181]:
my_inside_table = inside(sentence04, grammar2)

for row in my_inside_table:
    row.insert(0,{})
    print(row)

[{}, {'NP': -2.3025850929940455}, {}, {}, {'S': -4.633569660509789}, {}, {}, {'S': -13.325344836625804}]
[{}, {}, {'V': -0.10536051565782628}, {}, {'VP': -2.3309845675157437}, {}, {}, {'VP': -11.02275974363176}]
[{}, {}, {}, {'D': -0.6931471805599453}, {'NP': -1.7147984280919266}, {}, {}, {'NP': -5.039034768617953}]
[{}, {}, {}, {}, {'N': -0.5108256237659907}, {}, {}, {}]
[{}, {}, {}, {}, {}, {'P': 0.0}, {}, {'PP': -2.120263536200091}]
[{}, {}, {}, {}, {}, {}, {'D': -0.6931471805599453}, {'NP': -2.120263536200091}]
[{}, {}, {}, {}, {}, {}, {}, {'N': -0.916290731874155, 'V': -2.3025850929940455}]


In [178]:
yt_inside_table = inside(sentence03, grammar3)

for row in yt_inside_table:
    row.insert(0,{})
    print(row)

[{}, {'NP': -2.3025850929940455}, {}, {'S': -4.374058465024705}, {}, {'S': -11.995392229439306}]
[{}, {}, {'NP': -3.2188758248682006, 'V': 0.0}, {'VP': -2.071473372030659}, {}, {'VP': -9.69280713644526}]
[{}, {}, {}, {'NP': -1.7147984280919266}, {}, {'NP': -4.345887588058008}]
[{}, {}, {}, {}, {'P': 0.0}, {'PP': -1.7147984280919266}]
[{}, {}, {}, {}, {}, {'NP': -1.7147984280919266}]


In [170]:
def outside(sentence, grammar, beta):
    """
    Perform the outside algorithm

    :param sentence: The sentence to parse
    :param grammar: A dictionary representing the CNF grammar, with the key being the LHS and the value being a list of possible RHS.
    :return: A nxn CKY Table, with each cell containing the max possible log probability
    :param beta: the nxn table returned from the inside algorithm
    """
    #split sentence into words
    words = sentence.split()
    n = len(words)
    
    # Initialize the table to be nxn
    alpha = [[{} for j in range(n+1)] for i in range(n)]

    alpha[0][n]['S'] = 0  # 'S' is the start symbol

    for width in range(n, 0 , -1):
        for start in range(0, n - width + 1):
            end = start + width
            for mid in range(start + 1, end):

                for X, rules in grammar.items():
                    for rhs, prob in rules:
                        if len(rhs) == 2: 
                            Y, Z = rhs
                            # alpha[Y][start][mid]
                            if Y in beta[start][mid]:
                                if Y not in alpha[start][mid]:
                                    alpha[start][mid][Y] = 0.0
                                alpha[start][mid][Y] += alpha[start][end].get(X, 0.0) + beta[mid][end].get(Z, 0.0) + math.log(prob)
                            
                            # alpha[Z][mid][end]
                            if Z in beta[mid][end]:
                                if Z not in alpha[mid][end]:
                                    alpha[mid][end][Z] = 0.0
                                alpha[mid][end][Z] += alpha[start][end].get(X, 0.0) + beta[start][mid].get(Y, 0.0) + math.log(prob)

    return alpha

In [171]:
table = outside(sentence04, grammar2 ,my_inside_table)

for row in table:
    print(row)

[{}, {'NP': -20.577581137103117}, {}, {}, {}, {}, {}, {'S': 0}]
[{}, {}, {'V': -21.084852326344528}, {}, {'VP': -9.474305917810646}, {}, {}, {'VP': -2.3025850929940455}]
[{}, {}, {}, {'D': -25.746821461376015}, {'NP': -19.76309648636221}, {}, {}, {'NP': -3.429596856183853}]
[{}, {}, {}, {}, {'N': -21.988720538220125}, {}, {}, {}]
[{}, {}, {}, {}, {}, {'P': -20.379282625786026}, {}, {'PP': -18.259019089585934}]
[{}, {}, {}, {}, {}, {}, {'D': -24.54284865705008}, {'NP': -23.11573230140993}]
[{}, {}, {}, {}, {}, {}, {}, {'N': -30.30343008074967}]
